In [8]:
infile = open('day18_input.txt', 'r')
data_lines = []
for line in infile:
    data_lines.append(line[:-1])

infile.close()

In [ ]:
import random

class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))
    
    #@classmethod
    #def from_vec2f(cls, vf):
        #return cls(int(round(vf.x)), int(round(vf.y)))

    @classmethod
    def from_index(cls, ind):
        ssplit = ind[1:].split('P')
        yoff = 1
        if(len(ssplit) == 1):
            ssplit = ind[1:].split('N')
            yoff = -1
        
        xoff = 1
        if(ind[0] == 'N'):
            xoff = -1
        
        return cls(xoff * int(ssplit[0]), yoff * int(ssplit[1]))
        
    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def diff(self, v2):
        return vec2i(v2.x - self.x, v2.y - self.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))

def rev_dir(forward):
    if(forward == 1):
        return 2
    elif(forward == 2):
        return 1
    elif(forward == 3):
        return 4
    elif(forward == 4):
        return 3
    
    
    
def move_steps(idir):
    return {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}[idir]
    
class Intersection(object):
    def __init__(self, pos):
        self.pos = pos
        self.paths = {} #direction -> path
        self.pathstart = {} #whether this intersection is the start of given path or end
        
    def add_path(self, idir, path):
        self.paths[idir] = path
        self.pathstart[idir] = (path.start == self)
        

class Path(object):
    def __init__(self, start, firststep, current_endpt, firstchar):
        self.steps = [firststep]
        self.stepdirs = [] #first dir is steps[0] -> steps[1] (since intersection includes 
                           #direction), last is steps[n] -> end_intersection
        self.locks = {} #lock -> pos
        self.keys = {} #key -> pos
        self.length = 1
        self.start = start #intersection
        self.end = None
        self.current_endpt = current_endpt
        self.is_loop = False #when a path starts and ends at the same intersection
        self.dead_end = False
        self.register_step_char(firststep, firstchar)
        
    def add_step(self, step, stepdir, char = '.'):
        self.steps.append(step)
        self.stepdirs.append(stepdir)
        self.length = self.length + 1
        self.register_step_char(step, char)
    
    def register_step_char(self, step, char):
        ai = ord(char)
        if(ai >= ord('A') and ai <= ord("Z")):
            self.locks[chr(ai+32))] = step
        if(ai >= ord('a') and ai <= ord("z")):
            self.keys[char] = step
            
    def end_path(self, end, idir):
        
        #case of a "room", ie two adjacent intersections
        if self.steps[0] == end:
            self.steps = []
            self.stepdirs = []
            self.length = 0
        self.end = end
        self.stepdirs.append(idir)
        self.is_loop = (self.end == self.start)
        self.end.add_path(rev_dir(idir), self)
        
    def mark_dead_end(self):
        self.dead_end = True

    
class Area(object):
    def __init__(self, init_data):
        self.map = {}
        self.width = len(init_data[0])
        self.height = len(init_data)
        self.data = init_data
        self.intersections = {} #pos to intersection
        self.paths = {} #pos to path
        self.entrance = None

        entrancepos = None
        for y in range(1, self.height - 1):
            for x in range(1, self.width - 1):
                char = self.data[y][x]
                if char == '#':
                    continue
                ind = vec2i(x,y).index()
                if char == '@':
                    self.entrance = ind
                    entrancepos = vec2i(x,y)
                outlets = 0
                if(self.data[y-1][x] != '#'):
                    outlets = outlets + 1
                if(self.data[y+1][x] != '#'):
                    outlets = outlets + 1
                if(self.data[y][x-1] != '#'):
                    outlets = outlets + 1
                if(self.data[y][x+1] != '#'):
                    outlets = outlets + 1
                if(outlets > 2):
                    self.intersections[ind] = Intersection(ind)
        
        #special case the entrance, if it's not a real intersection, make it one anyway
        if(not(self.entrance in intersections)):
            self.intersections[self.entrance] = Intersection(self.entrance)
            
        #todo flood, create paths. once a path has started, follow it until an intersection is found
        openlst = []
        visited = set()
        visited.add(self.entrance)
        for idir in range(1,5):
            testpos = entrancepos.step(move_steps(idir))
            if(self.data[testpos.y][testpos.x] == '#'):
                continue
            dirind = testpos.index()
            if dirind in visited:
                continue
            visited.add(dirind)
            newpath = Path(self.entrance, dirind, testpos, self.data[testpos.y][testpos.x])
            self.intersections[self.entrance].add_path(idir, newpath)
            openlst.append(newpath)
        
        while(len(openlst) > 0):
            cur_path = openlst[0]
            openlst = openlst[1:]
            
            startpos = cur_path.current_endpt
            startind = startpos.index()
            pathing = True
            while(pathing):
                oldind = None
                oldpos = None
                olddir = None
                newind = None
                newpos = None
                newdir = None
                for idir in range(1,5):
                    testpos = startpos.step(move_steps(idir))
                    if(self.data[testpos.y][testpos.x] == '#'):
                        continue
                    dirind = testpos.index()
                    if dirind in visited:
                        oldind = dirind
                        oldpos = testpos
                        olddir = idir
                        continue
                    if newind != None:
                        print("error, found multiple indexes in non-intersection %" % dirind)
                    else:
                        newind = dirind
                        newpos = testpos
                        newdir = idir

                #this should onlyl occur in the case of a loop (including the null case of a single step dead end)
                if newind == None:
                    cur_path.mark_dead_end()
                    pathing = False
                elif newind in self.intersections:
                    cur_path.end_path(newind, newdir)
                    if not(newind in visited):
                        
                        for idir in range(1,5):
                            testpos = newpos.step(move_steps(idir))
                            if(self.data[testpos.y][testpos.x] == '#'):
                                continue
                            dirind = testpos.index()
                            if dirind in visited:
                                continue
                            visited.add(dirind)
                            newpath = Path(newind, dirind, testpos, self.data[testpos.y][testpos.x])
                            self.intersections[newind].add_path(idir, newpath)
                            openlst.append(newpath)
                        
                    pathing = False
                else:
                    cur_path.add_step(newind, newdir, self.data[newpos.y][newpos.x])
                    startpos = newpos
                    startind = newind

                visited.add(newind)
        #going to need to handle loop paths with locks + keys (sequencing)
        #going to need to handle intersections with locks + keys
        
        #can we solve both by making all locks into intersections? keys?
      